In [1]:
##------------------Import Libraries --------------------------##
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [2]:
##------------------Read Data----------------------------------##
file_path = 'SIPP_analysis_data.xlsx'
data = pd.read_excel(file_path)

In [3]:
data.head()

unique_id        ssuid      repwgt0       repwgt1       repwgt2  \
0          1  11428546521  7426.116541   3272.646846   2984.807011   
1          2  11428577022  9260.630659  17036.969529  13129.978967   
2          3  11481674622  4837.305089   8031.548547   8130.963960   
3          4  12889229422  9638.013075  16219.670390  13163.080201   
4          5  13309398822  5190.990802   8106.156293   7452.235473   

        repwgt3       repwgt4       repwgt5       repwgt6       repwgt7  ...  \
0   4144.434128   3603.779767  12749.500402   3534.087316   3677.640142  ...   
1  13180.834760  15801.090740   4445.420139  12091.404756  12781.709130  ...   
2   7984.299255   7644.212702   2273.861069   7101.412235   7670.058779  ...   
3  12818.263387  12173.747482  13922.769296   5538.784662  14840.845358  ...   
4   9004.767596   2426.520981   2555.204602   7252.986835   9276.875285  ...   

   unsecured_debt  unable_pay  mortgage_payment  asset_values  bachelors  \
0               0           0              1390        430510          0   
1           40300           1              1300         41732          0   
2           20000           0               500        169050          1   
3               0           0                 0          6880          0   
4           15000           0              1200           400          0   

   years_since_movein  unemployed  out_lf  born_abroad  likely_retired  
0         -140.416672           0       0            1               0  
1         -163.166672           0       0            0               0  
2         -130.666672           0       1            0               1  
3         -165.333328           0       0            0               0  
4                 NaN           0       1            0               0  

[5 rows x 324 columns]

In [4]:
data.isnull().sum()

unique_id                0
ssuid                    0
repwgt0                  0
repwgt1                  0
repwgt2                  0
                      ... 
years_since_movein    2747
unemployed               0
out_lf                   0
born_abroad              0
likely_retired           0
Length: 324, dtype: int64

In [5]:
data.dtypes

unique_id               int64
ssuid                   int64
repwgt0               float64
repwgt1               float64
repwgt2               float64
                       ...   
years_since_movein    float64
unemployed              int64
out_lf                  int64
born_abroad             int64
likely_retired          int64
Length: 324, dtype: object

In [6]:
data = data.drop('years_since_movein', axis=1)

In [7]:
data.columns

Index(['unique_id', 'ssuid', 'repwgt0', 'repwgt1', 'repwgt2', 'repwgt3',
       'repwgt4', 'repwgt5', 'repwgt6', 'repwgt7',
       ...
       'secured_debt', 'unsecured_debt', 'unable_pay', 'mortgage_payment',
       'asset_values', 'bachelors', 'unemployed', 'out_lf', 'born_abroad',
       'likely_retired'],
      dtype='object', length=323)

In [8]:
## Display all columns
pd.set_option('display.max_columns', None)

In [ ]:
##---------code not necessary for now--------------##
#print(data.columns.tolist())

['unique_id', 'ssuid', 'repwgt0', 'repwgt1', 'repwgt2', 'repwgt3', 'repwgt4', 'repwgt5', 'repwgt6', 'repwgt7', 'repwgt8', 'repwgt9', 'repwgt10', 'repwgt11', 'repwgt12', 'repwgt13', 'repwgt14', 'repwgt15', 'repwgt16', 'repwgt17', 'repwgt18', 'repwgt19', 'repwgt20', 'repwgt21', 'repwgt22', 'repwgt23', 'repwgt24', 'repwgt25', 'repwgt26', 'repwgt27', 'repwgt28', 'repwgt29', 'repwgt30', 'repwgt31', 'repwgt32', 'repwgt33', 'repwgt34', 'repwgt35', 'repwgt36', 'repwgt37', 'repwgt38', 'repwgt39', 'repwgt40', 'repwgt41', 'repwgt42', 'repwgt43', 'repwgt44', 'repwgt45', 'repwgt46', 'repwgt47', 'repwgt48', 'repwgt49', 'repwgt50', 'repwgt51', 'repwgt52', 'repwgt53', 'repwgt54', 'repwgt55', 'repwgt56', 'repwgt57', 'repwgt58', 'repwgt59', 'repwgt60', 'repwgt61', 'repwgt62', 'repwgt63', 'repwgt64', 'repwgt65', 'repwgt66', 'repwgt67', 'repwgt68', 'repwgt69', 'repwgt70', 'repwgt71', 'repwgt72', 'repwgt73', 'repwgt74', 'repwgt75', 'repwgt76', 'repwgt77', 'repwgt78', 'repwgt79', 'repwgt80', 'repwgt81', 're

In [9]:
num_column = data.select_dtypes(include=['int64', 'float64'])
cat_column = data.select_dtypes(include='object')

#### Data Preparation

In [10]:
###--------------------find replicate weights----------------------###
repl_weights = [col for col in num_column.columns if 'repwgt' in col]

In [24]:
###--------------------find features-----------------------------###
# Assuming that our target variable is 'asset_values'
# this is only to showcase how to model the replicate weights
target = 'asset_values'
features =  [col for col in num_column.columns if col not in repl_weights and col != target and 'area_state' not in col]


In [12]:
###-------------------Primary Weights---------------------------###
primary_weight = 'wpfinwgt'

In [14]:
num_column.isnull().sum()

unique_id         0
ssuid             0
repwgt0           0
repwgt1           0
repwgt2           0
                 ..
bachelors         0
unemployed        0
out_lf            0
born_abroad       0
likely_retired    0
Length: 317, dtype: int64

In [15]:
##------Show columns with only NaN---------###
nan_columns = num_column.columns[num_column.isnull().any()]
num_column_nan = num_column[nan_columns]

In [21]:
num_column.head()

unique_id        ssuid      repwgt0       repwgt1       repwgt2  \
0          1  11428546521  7426.116541   3272.646846   2984.807011   
1          2  11428577022  9260.630659  17036.969529  13129.978967   
2          3  11481674622  4837.305089   8031.548547   8130.963960   
3          4  12889229422  9638.013075  16219.670390  13163.080201   
4          5  13309398822  5190.990802   8106.156293   7452.235473   

        repwgt3       repwgt4       repwgt5       repwgt6       repwgt7  \
0   4144.434128   3603.779767  12749.500402   3534.087316   3677.640142   
1  13180.834760  15801.090740   4445.420139  12091.404756  12781.709130   
2   7984.299255   7644.212702   2273.861069   7101.412235   7670.058779   
3  12818.263387  12173.747482  13922.769296   5538.784662  14840.845358   
4   9004.767596   2426.520981   2555.204602   7252.986835   9276.875285   

        repwgt8       repwgt9      repwgt10      repwgt11      repwgt12  \
0   3230.083912   4260.458057   3552.429017  11200.324897   3889.409995   
1  13080.026440  12789.763710  12873.072834   4626.026005  11777.663138   
2   7093.145829   6849.969060   7869.893102   2438.539060   7133.519369   
3  12442.508911  12403.304557  13895.287063  17604.141394   3930.148083   
4   2883.200450   7162.646132   2171.803471   2447.723128   8123.634136   

       repwgt13      repwgt14      repwgt15      repwgt16      repwgt17  \
0   3528.610259  13421.481024  11755.565803  11136.988055  11432.845886   
1  13516.963689   5096.202394   4380.241439   5139.238464   4313.991866   
2   7480.541761   2440.850178   2764.321036   2667.431813   2193.680352   
3  12461.288686  13714.951744   4789.805641   6189.394182   4428.507194   
4   2778.252238   7449.035822   7291.355467   7236.298679   2555.003163   

       repwgt18      repwgt19      repwgt20      repwgt21      repwgt22  \
0   3220.478567  10791.684852   3495.716444  11561.986201   3145.321874   
1  13457.212033   5864.751281  13091.712112   4515.418135  14642.778149   
2   7979.962702   2556.193687   7124.952452   2579.766510   8418.236168   
3   5087.601104  13769.536134   3671.302668  13690.745909   6016.055516   
4   2731.109974   2834.478672   2611.199922   2400.257832   2590.663008   

       repwgt23      repwgt24      repwgt25      repwgt26      repwgt27  \
0   3505.917350   9467.251356  10998.052770   3848.153385   4294.853048   
1  10323.740957   4520.982427   4649.661321  12360.283205  16949.706262   
2   7785.253544   2349.094762   2457.517866   6895.779027   7789.456924   
3  15262.141078  16335.153997   5389.505034   5077.897351  15738.846478   
4   6028.010866   7437.627295   7625.099387   3046.505890   8389.604977   

       repwgt28      repwgt29      repwgt30      repwgt31      repwgt32  \
0  11621.391137   3780.624235   3735.315147  10850.186954  10524.711884   
1   4526.383837  14591.596986  12558.134163   4278.313123   4700.908312   
2   2797.940322   8012.602903   6969.726158   2497.439389   2545.099026   
3  14523.359994   5868.193129  14085.910165  13124.550286   6061.771161   
4   3609.548992   3003.715933   7928.013545   2668.143313   6829.312323   

       repwgt33      repwgt34      repwgt35      repwgt36      repwgt37  \
0  10471.912780   3407.040016   4233.630372  10269.943142   3745.280619   
1   4715.574499  10046.212701  13428.449395   4224.459877  14316.672410   
2   2690.728250   6871.893240   7847.783356   3081.769862   7604.697649   
3   6469.104890   4188.229516  15516.482263  17927.552870   5263.279869   
4   7920.676340   2825.503900   2302.481150   6362.908977   8538.050515   

       repwgt38      repwgt39      repwgt40      repwgt41      repwgt42  \
0   4371.216771   9911.347834  10096.168623   3653.441990   3322.757928   
1  14778.781592   4982.705695   3891.093767  13266.864353  13650.759435   
2   7792.603772   2344.429479   2991.041119   7451.702961   8768.817022   
3  14889.415435  18935.146302   4926.494350   4923.732534  13695.219353   
4   6939.910803   2958.177152   3262.626719   2402

In [20]:
num_column_nan

area_state1  area_state2  area_state4  area_state5  area_state6  \
0              NaN          NaN          NaN          NaN          NaN   
1              NaN          NaN          NaN          NaN          NaN   
2              NaN          NaN          NaN          NaN          NaN   
3              NaN          NaN          NaN          NaN          NaN   
4              NaN          NaN          NaN          NaN          NaN   
...            ...          ...          ...          ...          ...   
17433          NaN          NaN          NaN          NaN          1.0   
17434          NaN          NaN          NaN          NaN          1.0   
17435          NaN          NaN          NaN          NaN          1.0   
17436          NaN          NaN          NaN          NaN          1.0   
17437          NaN          NaN          NaN          NaN          1.0   

       area_state8  area_state9  area_state10  area_state11  area_state12  \
0              NaN          NaN           NaN           NaN           NaN   
1              NaN          NaN           NaN           NaN           NaN   
2              NaN          NaN           NaN           NaN           NaN   
3              NaN          NaN           NaN           NaN           NaN   
4              NaN          NaN           NaN           NaN           NaN   
...            ...          ...           ...           ...           ...   
17433          NaN          NaN           NaN           NaN           NaN   
17434          NaN          NaN           NaN           NaN           NaN   
17435          NaN          NaN           NaN           NaN           NaN   
17436          NaN          NaN           NaN           NaN           NaN   
17437          NaN          NaN           NaN           NaN           NaN   

       area_state13  area_state15  area_state16  area_state17  area_state18  \
0               NaN           NaN           NaN           NaN           NaN   
1               NaN           NaN           NaN           NaN           NaN   
2               NaN           NaN           NaN           NaN           NaN   
3               NaN           NaN           NaN           NaN           NaN   
4               NaN           NaN           NaN           NaN           NaN   
...             ...           ...           ...           ...           ...   
17433           NaN           NaN           NaN           NaN           NaN   
17434           NaN           NaN           NaN           NaN           NaN   
17435           NaN           NaN           NaN           NaN           NaN   
17436           NaN           NaN           NaN           NaN           NaN   
17437           NaN           NaN           NaN           NaN           NaN   

       area_state19  area_state20  area_state21  area_state22  area_state23  \
0               NaN           1.0           NaN           NaN           NaN   
1               NaN           1.0           NaN           NaN           NaN   
2               NaN           1.0           NaN           NaN           NaN   
3               NaN           NaN           NaN           NaN           NaN   
4               NaN           NaN           NaN           NaN           NaN   
...             ...           ...           ...           ...           ...   
17433           NaN           NaN           NaN           NaN           NaN   
17434           NaN           NaN           NaN           NaN           NaN   
17435           NaN           NaN           NaN           NaN           NaN   
17436           NaN           NaN           NaN           NaN           NaN   
17437           NaN           NaN           NaN           NaN           NaN   

       area_state24  area_state25  area_state26  area_state27  area_state28  \
0               NaN           NaN           NaN           NaN           NaN   
1               NaN           NaN           NaN           NaN           NaN   
2               NaN           NaN           NaN           

In [17]:
##---------Exclude the columns with NaN values----------------##
num_column_clean = num_column.dropna(axis=1, inplace=True)

In [22]:
num_column.isnull().sum()

unique_id         0
ssuid             0
repwgt0           0
repwgt1           0
repwgt2           0
                 ..
bachelors         0
unemployed        0
out_lf            0
born_abroad       0
likely_retired    0
Length: 266, dtype: int64

In [ ]:
###-----------------Fit data on Primary Weight-----------------###

X = num_column[features]
y = num_column[target]
sample_weight = num_column[primary_weight]

###------------------Split Data-------------------------------###
X_train, X_test, y_train, y_test, weights_train, weights_test = train_test_split(X, y, sample_weight, test_size=0.3, random_state=42)

model = LinearRegression()
"Extra Coding process needed if doing cross-validation, or other hyperparametization"
"Might have to make process changes based on a different model."
model.fit(X_train, y_train, sample_weight=weights_train)

# Evaluate the model
# predict on the test set
baseline_predictions = model.predict(X_test)
baseline_mse = mean_squared_error(y_test, baseline_predictions)

print(f'Baseline Model MSE: {baseline_mse}')
print(f'Model Coefficients: {model.coef_}')
print(f'Intercept: {model.intercept_}')

Baseline Model MSE: 1.3619245732091876e-17
Model Coefficients: [ 8.36045858e-15  8.27180613e-25  5.24943158e-14  1.00000000e+00
  0.00000000e+00 -8.62624972e-16  5.07694077e-13 -5.17836841e-12
  3.22429557e-12 -1.26186026e-11  1.29309160e-10  4.98543644e-16
  3.25362982e-16  2.37011237e-16  1.41619216e-15  1.00000000e+00
  1.00000000e+00  4.59098527e-12 -3.15066900e-17  2.11580761e-11
  7.79448002e-12 -1.37981622e-11 -2.91155665e-11 -2.45205092e-11]
Intercept: 2.3283064365386963e-10


In [39]:
weight_sample.shape

(12206,)

In [40]:
X_train.shape

(12206, 24)

In [41]:
num_column.shape

(17438, 266)

In [42]:
12206/17438

0.6999655923844478

In [ ]:
###-----------------Fit on Replicate Weights-----------------------###
repl_prod = []

for weight in repl_weights:
    weight_sample = num_column[weight]
    # check for mismatch between weight and sample shape
    if weight_sample.shape[0] != X_train.shape[0]:
        print(f'Weight sample for {weight} has a mismatch in the number of samples')
        weight_sample = weight_sample[:X_train.shape[0]]
    model.fit(X_train, y_train, sample_weight=weight_sample)
    predictions = model.predict(X_test)
    repl_prod.append(predictions)

# convert to dataframe
repl_prod = pd.DataFrame(repl_prod).T

Weight sample for repwgt0 has a mismatch in the number of samples
Weight sample for repwgt1 has a mismatch in the number of samples
Weight sample for repwgt2 has a mismatch in the number of samples
Weight sample for repwgt3 has a mismatch in the number of samples
Weight sample for repwgt4 has a mismatch in the number of samples
Weight sample for repwgt5 has a mismatch in the number of samples
Weight sample for repwgt6 has a mismatch in the number of samples
Weight sample for repwgt7 has a mismatch in the number of samples
Weight sample for repwgt8 has a mismatch in the number of samples
Weight sample for repwgt9 has a mismatch in the number of samples
Weight sample for repwgt10 has a mismatch in the number of samples
Weight sample for repwgt11 has a mismatch in the number of samples
Weight sample for repwgt12 has a mismatch in the number of samples
Weight sample for repwgt13 has a mismatch in the number of samples
Weight sample for repwgt14 has a mismatch in the number of samples
Weigh

In [37]:
# calculate variance across replicate predictions
prediction_variance = repl_prod.var(axis=1)

# Average predictions across replicate models for final estimate.
final_predictions = repl_prod.mean(axis=1)

print(f'Average Predictions for replicate weights: \n{final_predictions}')

Average Predictions for replicate weights: 
0         392840.0
1         328500.0
2           1005.0
3         224860.0
4         246400.0
           ...    
5227      713301.0
5228    12522080.0
5229      355500.0
5230       30980.0
5231      149550.0
Length: 5232, dtype: float64


In [ ]:
repl_weights = ['repwgt0', 'repwgt1', 'repwgt2', 'repwgt3', 'repwgt4', 'repwgt5', 'repwgt6', 'repwgt7', 'repwgt8', 'repwgt9', 'repwgt10', 'repwgt11', 'repwgt12', 'repwgt13', 'repwgt14', 'repwgt15', 'repwgt16', 
            'repwgt17', 'repwgt18', 'repwgt19', 'repwgt20', 'repwgt21', 'repwgt22', 'repwgt23', 'repwgt24', 'repwgt25', 'repwgt26', 'repwgt27', 'repwgt28', 'repwgt29', 'repwgt30', 'repwgt31', 'repwgt32', 'repwgt33', 
            'repwgt34', 'repwgt35', 'repwgt36', 'repwgt37', 'repwgt38', 'repwgt39', 'repwgt40', 'repwgt41', 'repwgt42', 'repwgt43', 'repwgt44', 'repwgt45', 'repwgt46', 'repwgt47', 'repwgt48', 'repwgt49', 
            'repwgt50', 'repwgt51', 'repwgt52', 'repwgt53', 'repwgt54', 'repwgt55', 'repwgt56', 'repwgt57', 'repwgt58', 'repwgt59', 'repwgt60', 'repwgt61', 'repwgt62', 'repwgt63', 'repwgt64', 'repwgt65', 
            'repwgt66', 'repwgt67', 'repwgt68', 'repwgt69', 'repwgt70', 'repwgt71', 'repwgt72', 'repwgt73', 'repwgt74', 'repwgt75', 'repwgt76', 'repwgt77', 'repwgt78', 'repwgt79', 'repwgt80', 'repwgt81', 
            'repwgt82', 'repwgt83', 'repwgt84', 'repwgt85', 'repwgt86', 'repwgt87', 'repwgt88', 'repwgt89', 'repwgt90', 'repwgt91', 'repwgt92', 'repwgt93', 'repwgt94', 'repwgt95', 'repwgt96', 'repwgt97', 
            'repwgt98', 'repwgt99', 'repwgt100', 'repwgt101', 'repwgt102', 'repwgt103', 'repwgt104', 'repwgt105', 'repwgt106', 'repwgt107', 'repwgt108', 'repwgt109', 'repwgt110', 'repwgt111', 'repwgt112', 
            'repwgt113', 'repwgt114', 'repwgt115', 'repwgt116', 'repwgt117', 'repwgt118', 'repwgt119', 'repwgt120', 'repwgt121', 'repwgt122', 'repwgt123', 'repwgt124', 'repwgt125', 'repwgt126', 'repwgt127', 
            'repwgt128', 'repwgt129', 'repwgt130', 'repwgt131', 'repwgt132', 'repwgt133', 'repwgt134', 'repwgt135', 'repwgt136', 'repwgt137', 'repwgt138', 'repwgt139', 'repwgt140', 'repwgt141', 'repwgt142', 
            'repwgt143', 'repwgt144', 'repwgt145', 'repwgt146', 'repwgt147', 'repwgt148', 'repwgt149', 'repwgt150', 'repwgt151', 'repwgt152', 'repwgt153', 'repwgt154', 'repwgt155', 'repwgt156', 'repwgt157', 
            'repwgt158', 'repwgt159', 'repwgt160', 'repwgt161', 'repwgt162', 'repwgt163', 'repwgt164', 'repwgt165', 'repwgt166', 'repwgt167', 'repwgt168', 'repwgt169', 'repwgt170', 'repwgt171', 'repwgt172', 
            'repwgt173', 'repwgt174', 'repwgt175', 'repwgt176', 'repwgt177', 'repwgt178', 'repwgt179', 'repwgt180', 'repwgt181', 'repwgt182', 'repwgt183', 'repwgt184', 'repwgt185', 'repwgt186', 'repwgt187', 
            'repwgt188', 'repwgt189', 'repwgt190', 'repwgt191', 'repwgt192', 'repwgt193', 'repwgt194', 'repwgt195', 'repwgt196', 'repwgt197', 'repwgt198', 'repwgt199', 'repwgt200', 'repwgt201', 'repwgt202', 
            'repwgt203', 'repwgt204', 'repwgt205', 'repwgt206', 'repwgt207', 'repwgt208', 'repwgt209', 'repwgt210', 'repwgt211', 'repwgt212', 'repwgt213', 'repwgt214', 'repwgt215', 'repwgt216', 'repwgt217', 
            'repwgt218', 'repwgt219', 'repwgt220', 'repwgt221', 'repwgt222', 'repwgt223', 'repwgt224', 'repwgt225', 'repwgt226', 'repwgt227', 'repwgt228', 'repwgt229', 'repwgt230', 'repwgt231', 'repwgt232', 
            'repwgt233', 'repwgt234', 'repwgt235', 'repwgt236', 'repwgt237', 'repwgt238', 'repwgt239', 'repwgt240']

In [ ]:

features_ = ['net_worth', 'area_US', 'race_eth_det', 'race_det5', 'race_eth', 'race_eth3', 'race_det3', 'poc', 'hh_inc', 'age', 
            'female', 'hh_size', 'married', 'homeowner', 'home_value', 
               'home_debt', 'student_debt', 'credit_card_debt', 'secured_debt', 'unsecured_debt', 'unable_pay', 'mortgage_payment',
               'asset_values', 'bachelors', 'years_since_movein', 'unemployed', 'out_lf', 'born_abroad', 'likely_retired']

In [ ]:
###-----------Better way to Extract Columns-------------------------###
# Extract replicate weight columns
repl_weights = [col for col in data.columns if 'repwgt' in col]

# Extract feature columns
features =  [col for col in num_column.columns if col not in repl_weights and col != target and 'area_state' not in col]